# **Intercomparison Demo**

**Author:** Adam Yang (ayang115@umd.edu)

**Description** Demo for Intercomparison Script: extracts test points, provided from hand labeled maps, from industry covermaps (Harvest, Copernicus, ESA, GLAD) and determines accuracy across maps.


In [16]:
# !earthengine authenticate

In [1]:
import ee
ee.Initialize()

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ee
from shapely import wkt

import sys
sys.path.append("..")
from src.compare_covermaps import *

c:\Users\adamf\anaconda3\envs\intercomparison\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
maps = [v for v in TARGETS.values()]
countries = ["Togo", "Kenya", "Malawi", "Tanzania"]

In [4]:
covermap_test = TestCovermaps(test_countries=countries, covermaps=maps)

# test_pts = pd.read_csv("../../intercomparison-data/extracted/test/tgo_tan_ken_2019_(newkenya).csv").drop("Unnamed: 0", axis=1)
test_pts = pd.read_csv("../../intercomparison-data/extracted/test/tgo_ken_tza_mlw_test.csv").drop("Unnamed: 0", axis=1)
test_pts["geometry"] = test_pts["geometry"].apply(wkt.loads)
test_pts = gpd.GeoDataFrame(test_pts, crs="epsg:4326")

# test_pts = covermap_test.get_test_points()

extracted = covermap_test.extract_covermaps(test_pts)
_ = covermap_test.evaluate()
results = pd.concat(covermap_test.results)
results


[Togo] sampling harvest-togo...
[Togo] sampling copernicus...
[Togo] sampling esa...
[Togo] sampling glad...
[Togo] sampling gfsad...
[Togo] sampling asap...
Extracting error: length of sampled dataset is not the same as testing dataset (less)
[Togo] sampling dynamicworld...
[Togo] sampling gfsad-gcep...
Extracting error: length of sampled dataset is not the same as testing dataset (more)
[Togo] sampling gfsad-lgrip...
[Togo] sampling de-africa-2019...
Extracting error: length of sampled dataset is not the same as testing dataset (less)
[Togo] sampling esa-cci-africa...
[Togo] sampling globcover-v23...
[Togo] sampling globcover-v22...
[Togo] sampling harvest-crop-maps...
[Togo] sampling esri-lulc...
[Kenya] sampling harvest-kenya...
Extracting error: length of sampled dataset is not the same as testing dataset (less)
[Kenya] sampling copernicus...
[Kenya] sampling esa...
[Kenya] sampling glad...
[Kenya] sampling gfsad...
[Kenya] sampling asap...
Extracting error: length of sampled data

c:\Users\adamf\anaconda3\envs\intercomparison\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\adamf\anaconda3\envs\intercomparison\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\adamf\anaconda3\envs\intercomparison\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

dataset: asap | country: Kenya
dataset: dynamicworld | country: Kenya
dataset: gfsad-gcep | country: Kenya
dataset: gfsad-lgrip | country: Kenya
dataset: de-africa-2019 | country: Kenya
dataset: esa-cci-africa | country: Kenya
dataset: globcover-v23 | country: Kenya
dataset: globcover-v22 | country: Kenya
dataset: harvest-crop-maps | country: Kenya
dataset: esri-lulc | country: Kenya
dataset: copernicus | country: Malawi
dataset: esa | country: Malawi
dataset: glad | country: Malawi
dataset: gfsad | country: Malawi
dataset: asap | country: Malawi
dataset: dynamicworld | country: Malawi
dataset: gfsad-gcep | country: Malawi
dataset: gfsad-lgrip | country: Malawi
dataset: de-africa-2019 | country: Malawi
dataset: esa-cci-africa | country: Malawi
dataset: globcover-v23 | country: Malawi
dataset: globcover-v22 | country: Malawi
dataset: harvest-crop-maps | country: Malawi


c:\Users\adamf\anaconda3\envs\intercomparison\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\adamf\anaconda3\envs\intercomparison\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\adamf\anaconda3\envs\intercomparison\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

dataset: esri-lulc | country: Malawi
dataset: harvest-tanzania | country: Tanzania
dataset: copernicus | country: Tanzania
dataset: esa | country: Tanzania
dataset: glad | country: Tanzania
dataset: gfsad | country: Tanzania
dataset: asap | country: Tanzania
dataset: dynamicworld | country: Tanzania
dataset: gfsad-gcep | country: Tanzania
dataset: gfsad-lgrip | country: Tanzania
dataset: de-africa-2019 | country: Tanzania
dataset: esa-cci-africa | country: Tanzania
dataset: globcover-v23 | country: Tanzania
dataset: globcover-v22 | country: Tanzania
dataset: esri-lulc | country: Tanzania


country  crop_f1  accuracy  crop_support  \
         dataset                                                        
Togo     harvest-togo           Togo     0.82      0.90            51   
         copernicus             Togo     0.00      0.72            51   
         esa                    Togo     0.70      0.86            51   
         glad                   Togo     0.71      0.86            51   
         gfsad                  Togo     1.00      1.00            51   
         asap                   Togo     0.47      0.42            45   
         dynamicworld           Togo     0.00      0.72            51   
         gfsad-gcep             Togo     0.62      0.78            51   
         gfsad-lgrip            Togo     0.59      0.77            51   
         de-africa-2019         Togo     0.68      0.86            43   
         esa-cci-africa         Togo     0.58      0.78            51   
         globcover-v23          Togo     0.43      0.74            51   
         globcover-v22          Togo     0.36      0.70            51   
         harvest-crop-maps      Togo     0.84      0.91            51   
         esri-lulc              Togo     0.07      0.73            51   
Kenya    harvest-kenya         Kenya     0.43      0.92            35   
         copernicus            Kenya     0.00      0.94            36   
         esa                   Kenya     0.41      0.94            36   
         glad                  Kenya     0.73      0.96            36   
         gfsad                 Kenya     1.00      1.00            36   
         asap                  Kenya     0.36      0.36            34   
         dynamicworld          Kenya     0.00      0.94            36   
         gfsad-gcep            Kenya     0.62      0.93            36   
         gfsad-lgrip           Kenya     0.62      0.93            36   
         de-africa-2019        Kenya     0.54      0.91            36   
         esa-cci-africa        Kenya     0.48      0.89            36   
         globcover-v23         Kenya     0.29      0.78            36   
         globcover-v22         Kenya     0.35      0.82            36   
         harvest-crop-maps     Kenya     0.43      0.92            35   
         esri-lulc             Kenya     0.66      0.96            36   
Malawi   copernicus           Malawi     0.00      0.80            31   
         esa                  Malawi     0.74      0.89            31   
         glad                 Malawi     0.62      0.86            31   
         gfsad                Malawi     1.00      1.00            31   
         asap                 Malawi     0.33      0.32            31   
         dynamicworld         Malawi     0.00      0.80            31   
         gfsad-gcep           Malawi     0.62      0.80            31   
         gfsad-lgrip          Malawi     0.63      0.80            31   
         de-africa-2019       Malawi     0.66      0.76            31   
         esa-cci-africa       Malawi     0.66      0.82            31   
         globcover-v23        Malawi     0.49      0.76            31   
         globcover-v22        Malawi     0.43      0.78            31   
         harvest-crop-maps    Malawi     0.82      0.92            31   
         esri-lulc            Malawi     0.44      0.82            31   
Tanzania harvest-tanzania   Tanzania     0.77      0.85           431   
         copernicus         Tanzania     0.00      0.64           431   
         esa                Tanzania     0.83      0.89           431   
         glad               Tanzania     0.78      0.87           431   
         gfsad              Tanzania     1.00      1.00           431   
         asap               Tanzania     0.74      0.61           389   
         dynamicworld       Tanzania     0.00      0.64           431   
         gfsad-gcep         Tanzania     0.82      0.88           431   
         gfsad-lgrip        Tanzania     0.83      0.89           431   
         de-af

In [6]:
# results.to_csv("../notebooks/intercomparison-results.csv")

In [ ]:
test_pts = pd.read_csv("../../intercomparison-data/extracted/test/tgo_ken_tza_mlw_test.csv").drop("Unnamed: 0", axis=1)
test_pts["geometry"] = test_pts["geometry"].apply(wkt.loads)
test_pts = gpd.GeoDataFrame(test_pts, crs="epsg:4326")

In [13]:
a =ee.ImageCollection("users/potapovpeter/Global_cropland_2019")

In [14]:
b = extract_points(a, test_pts, resolution=10)

In [15]:
b

,geometry,b1,binary,lat,lon
0,None,0,0,9.875907,1.172471
1,None,0,0,9.275314,1.196385
2,None,0,0,7.346575,0.705167
3,None,1,1,8.377827,1.275045
4,None,0,0,8.688244,1.388867
...,...,...,...,...,...
2105,None,1,0,-13.841996,33.322652
2106,None,0,0,-11.182818,33.466135
2107,None,0,0,-12.980817,33.978386
2108,None,0,0,-14.983120,35.745454


TODO:
1. reduce regions
2. add esri